<a href="https://colab.research.google.com/github/diaboloshogunate/GoogleColabML/blob/main/fashionMNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import torch
from torch import nn
import torchvision
from torchvision.transforms import ToTensor

In [14]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [15]:
# classes
classes = ('T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot')

In [16]:
# parameters
batch_size = 4
learning_rate = 0.001
momentum = 0
epochs = 2
loss_fn = nn.CrossEntropyLoss()
layers = [nn.Linear(28*28, 1024), nn.ReLU(), nn.Linear(1024, 10)]
stack = nn.Sequential(*layers)

In [17]:
# datasets
trainset = torchvision.datasets.FashionMNIST('./data', download=True, train=True, transform=ToTensor())
testset = torchvision.datasets.FashionMNIST('./data', download=True, train=False, transform=ToTensor())

In [18]:
# data loaders
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
shuffle=False, num_workers=2)

In [19]:
for X, y in testloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([4, 1, 28, 28])
Shape of y:  torch.Size([4]) torch.int64


In [20]:
# Define model
class NeuralNetwork(nn.Module):
    def __init__(self, stack):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = stack

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork(stack).to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=momentum)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=1024, bias=True)
    (1): ReLU()
    (2): Linear(in_features=1024, out_features=10, bias=True)
  )
)


In [21]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [23]:
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(trainloader, model, loss_fn, optimizer)
    test(testloader, model, loss_fn)
    break

Epoch 1
-------------------------------
loss: 1.283097  [    0/60000]
loss: 0.810538  [  400/60000]
loss: 1.018733  [  800/60000]
loss: 0.858893  [ 1200/60000]
loss: 1.078508  [ 1600/60000]
loss: 1.082562  [ 2000/60000]
loss: 1.023061  [ 2400/60000]
loss: 0.890148  [ 2800/60000]
loss: 1.230326  [ 3200/60000]
loss: 1.273936  [ 3600/60000]
loss: 0.528963  [ 4000/60000]
loss: 0.943175  [ 4400/60000]
loss: 0.955897  [ 4800/60000]
loss: 0.735017  [ 5200/60000]
loss: 1.112352  [ 5600/60000]
loss: 1.391302  [ 6000/60000]
loss: 0.826579  [ 6400/60000]
loss: 1.136470  [ 6800/60000]
loss: 0.931134  [ 7200/60000]
loss: 1.516579  [ 7600/60000]
loss: 0.469340  [ 8000/60000]
loss: 0.609148  [ 8400/60000]
loss: 0.459164  [ 8800/60000]
loss: 0.896766  [ 9200/60000]
loss: 0.505410  [ 9600/60000]
loss: 0.541379  [10000/60000]
loss: 0.597929  [10400/60000]
loss: 0.734848  [10800/60000]
loss: 1.661933  [11200/60000]
loss: 0.847861  [11600/60000]
loss: 0.736740  [12000/60000]
loss: 0.437764  [12400/60000]


KeyboardInterrupt: ignored